In [1]:
# default_exp data.finance

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from libs import is_lib_exists

In [4]:
required_libs = [ ("numpy", "numpy"),
                  ("pandas", "pandas"),
                  ("seaborn", "seaborn"),
                  ("matplotlib", "matplotlib"),
                  ("investpy", "investpy"),
                  ("dateutil", "python-dateutil"),
                ]

In [5]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
investpy exists
python-dateutil exists


In [88]:
#export
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import investpy
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
import calendar
import traceback

In [7]:
%matplotlib inline

In [8]:
# % - line, %% - cell
# %run, %%timeit, %%writefile, %pycat
# %who, %store
# %html 
# %env, %pinfo
# %lsmagic
# %history
# %prun
# %%heat
# https://github.com/lmmx/devnotes/wiki/IPython-'magic'-function-documentation#edit

In [9]:
from IPython.display import display, Image

# Finance Data

> Functionality to gather finance data.

The classes here provide bases classes used to gather finance data for model training.

In [10]:
#hide
from nbdev.showdoc import *

In [91]:
# Dataset path
DATASET_PATH = Path("../../data")  

# Country to analyze
COUNTRY = "malaysia"

# Dataset
DATASET = DATASET_PATH/f"ftse_klse_investing.csv"

# Stocks summary
DATASET_STOCKS_SUMMARY = DATASET_PATH/f"stocks_summary.csv"

## FTSE KLSE

In [12]:
# Retrieve stock components
dataset = investpy.get_stocks(country=COUNTRY)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    929 non-null    object
 1   name       929 non-null    object
 2   full_name  929 non-null    object
 3   isin       929 non-null    object
 4   currency   929 non-null    object
 5   symbol     929 non-null    object
dtypes: object(6)
memory usage: 43.7+ KB


In [13]:
# Save the dataset
dataset.to_csv(DATASET, index=False)

#dataset = pd.read_csv(DATASET, header=0, sep=',', quotechar='"')

In [14]:
display(dataset.head(3))

,country,name,full_name,isin,currency,symbol
0,malaysia,Media Chinese Int,Media Chinese International Ltd,BMG5959D1048,MYR,MDCH
1,malaysia,AMMB,AMMB Holdings Bhd,MYL1015OO006,MYR,AMMB
2,malaysia,CIMB Group,CIMB Group Holdings Bhd,MYL1023OO000,MYR,CIMB


In [15]:
# Today's date
TODAY = date.today()

# Past 3 years
PAST_3_YEARS = [ TODAY.year - y for y in range(1,4) ]

# Expected YIELD %
EXPECTED_YIELD_PERCENTAGE = 6

In [96]:
def calculate_yield(idx, name, symbol):
    """Calculate stock yield based on latest close price"""
    try:
        df_dividends = investpy.get_stock_dividends(stock=symbol, country=COUNTRY)
        #display(df_dividends)
        df_dividends['Year'] = pd.DatetimeIndex(df_dividends['Date']).year
        # Convert yield percentage to number
        df_dividends['Historical Yield'] =  df_dividends['Yield'].apply(\
                                                     lambda y : \
                                                     float(y.translate(str.maketrans({',': '.', '%': '', '-':'0'}))))
        
        # Aggregate the dividends by years
        df_dividends_summary = df_dividends.groupby(by=['Year'], sort=False, as_index=False).\
                                agg({'Dividend':'sum', 'Historical Yield':'sum', \
                                     'Date': lambda d: ','.join(d.dt.strftime('%Y-%m-%d'))})
        # print(df_dividends.groupby(by=['Year'], sort=False, as_index=False))
        # display(df_dividends_summary[df_dividends_summary['Year'].isin(PAST_3_YEARS)])
        #display(df_dividends_summary)
        
        df_recent_data = investpy.get_stock_recent_data(stock=symbol, country=COUNTRY).reset_index()
        last_close_price = df_recent_data[df_recent_data['Date'].eq(df_recent_data['Date'].max())]['Close'].values[0]
        # display(f"Recent price {last_close_price}")
        # display(df_recent_data)
     
        df_dividends_summary['Name'] = name
        df_dividends_summary['Symbol'] = symbol
        df_dividends_summary['Last Close Price'] = last_close_price
        df_dividends_summary['Calculated Yield'] = df_dividends_summary['Dividend'] / last_close_price * 100
       
        # display(df_dividends_summary)
        
        # df_desired_returns = df_dividends_summary[
        #                     (df_dividends_summary['Year'].isin(PAST_3_YEARS)) & 
        #                     (df_dividends_summary['Yield'] >= EXPECTED_YIELD_PERCENTAGE)]
        # if len(df_desired_returns) == len(PAST_3_YEARS):
        #     display(f"{name} - {symbol} met expected yield")
        #     display(df_desired_returns)
        
        print(f'{idx}. {symbol} - {name}')
        return df_dividends_summary
            
    except Exception as exception:
        display(f"Skipping {name} - {symbol}")
        traceback.print_exc()

def summarize_stocks(df_summary, df):
    """Summarize the stock dividends details"""
    if df_summary.empty:
        return df
    return df_summary.append(df)
    
    
df_all_dividends = pd.DataFrame()
START_INDEX = 0
#df = calculate_yield("MBSS", "MBSS")
#df_all_dividends = summarize_stocks(df_all_dividends, df)
for idx, row in dataset.iloc[START_INDEX:].iterrows():
    name = row['name']
    symbol = row['symbol']
    df = calculate_yield(idx, name, symbol)
    df_all_dividends = summarize_stocks(df_all_dividends, df)
    if (idx == 3):
        break

# display(df_all_dividends)
df_all_dividends.to_csv(DATASET_STOCKS_SUMMARY, index=False)

0. MDCH - Media Chinese Int
1. AMMB - AMMB
2. CIMB - CIMB Group
3. RHBC - RHB Bank
